In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# !wget https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin
# !wget https://huggingface.co/roberta-base/resolve/main/merges.txt
# !wget https://huggingface.co/roberta-base/resolve/main/vocab.json
# !wget https://huggingface.co/roberta-base/resolve/main/config.json

In [3]:
!pip install transformers
!pip install sentencepiece
bert_dir = '/content/gdrive/MyDrive/roberta-base'
# bert_dir = '.'
data_dir = '/content/gdrive/MyDrive/530project/Liar-Plus'
output_dir = '.'
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, AdamW, get_cosine_schedule_with_warmup
import warnings
warnings.filterwarnings("ignore")
from torch.utils.data import TensorDataset,DataLoader
# !pip install datasets
import pandas as pd
import argparse
import numpy as np
from collections import Counter
# from datasets import load_dataset
import os
import torch
import pickle
import re
import time
import copy
from torch.utils.data import DataLoader, Dataset
import torch.optim as optimizer 
from torch import nn
from sklearn.metrics import accuracy_score

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.sans-serif'] = ['Times New Roman']
sns.set_style("whitegrid")
sns.set_style({'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set(font_scale=1.2)
import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("--local_rank", type=int)
# args = parser.parse_args()
# torch.cuda.set_device(args.local_rank)
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
max_len = 28
max_len_meta = 132
BATCH_SIZE = 4
EPOCHS = 11
# LEARNING_RATE = 0.0001
LEARNING_RATE = 0.00008

     |████████████████████████████████| 3.3 MB 5.3 MB/s 
     |████████████████████████████████| 61 kB 462 kB/s 
     |████████████████████████████████| 895 kB 66.3 MB/s 
     |████████████████████████████████| 3.3 MB 22.5 MB/s 
     |████████████████████████████████| 596 kB 63.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [4]:
tokenizer = RobertaTokenizer.from_pretrained(bert_dir)
config = RobertaConfig.from_pretrained(bert_dir)
bertmodel = RobertaModel.from_pretrained(bert_dir)

Some weights of the model checkpoint at /content/gdrive/MyDrive/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def clean_text(w):
  if type(w)==float:
    return " "
  return re.sub(
          r"([.,'!?\"()*#:;])",
          '',
          w.lower()
          ).replace('/', ' ')
def preprocessing(train=False, eval=False, test=False):
  cols = ['id','label','statement','subject','speaker','job_title','state_info',
          'party_affiliation','barely_true_counts','false_counts',
          'half_true_counts','mostly_true_counts','pants_on_fire_counts',
          'context','justification']
  label_dict = {"false" : 0, "half-true" : 1, "mostly-true" : 2, "true": 3, "barely-true" : 4, "pants-fire" : 5 } 
  def get_label(x):
    if x not in label_dict:
      return 1
    return label_dict[x]
  if train:
    dst_path = os.path.join(data_dir,'train2.tsv')
  if eval:
    dst_path = os.path.join(data_dir,'val2.tsv')
  if test:
    dst_path = os.path.join(data_dir,'test2.tsv')
  current_dataset = pd.read_csv(dst_path, sep='\t', header = None, names=cols)
  current_dataset['label'] = current_dataset['label'].apply(lambda x: get_label(x))
  current_dataset.reset_index(drop=True,inplace=True)
  return current_dataset

meta_cols = ['subject','speaker','job_title','state_info','party_affiliation','context','justification']
def get_meta_embed(dst, meta_cols):
  all_text = []
  for i in range(len(dst)):
    cur = ''
    for c in meta_cols:
      try:
        cur += str(dst[c][i]) + ' SS '
      except:
        print(c,i)
        return
    all_text.append(cur)
  return all_text

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
def evaluate_p_r_f1_acc(y_pred, y_true):
    precision = precision_score(y_pred, y_true)
    recall = recall_score(y_pred, y_true)
    fscore = f1_score(y_pred, y_true)
    acc = accuracy_score(y_pred, y_true)
    return precision, recall, fscore, acc
def category_from_output(output):
  res = []
  for i in output:
    top_n, top_i = i.topk(1)
    category_i = top_i[0].item()
    res.append(category_i)
  return res


def evaluate(model,data_loader,device,name='cloth_bert.pth'):
    model.eval()
    val_true,val_pred = [],[]
    for idx,(ids,att,y,idm,attm) in enumerate(data_loader):
        # print(device)
        ids,att,y,idm,attm = ids.to(device),att.to(device),y.to(device),idm.to(device),attm.to(device)
        ids = torch.concat((ids,idm),dim=1)
        att = torch.concat((att,attm),dim=1)
    # for idx,(ids,tpe,att,y,meta) in enumerate(data_loader):
        # y_pred = model(ids.cuda(),tpe.cuda(),att.cuda())  # prob mat
        y_pred = model(ids, att)
        categories = category_from_output(y_pred)
        # loss = criteon(output,label)
        val_pred += categories
        val_true += y.tolist()
        if idx==0:
          print(val_pred,val_true)
    # p, r, fscore, acc = evaluate_p_r_f1_acc(val_pred, val_true)
    # print('\tF-score: ', fscore, '\tacc: ', acc)
    # return acc #, val_pred
    acc = accuracy_score(val_pred,val_true)#, f1_score(y_pred,y_true)
    print(Counter(val_pred))
    print('acc: ', acc)
    return acc

In [6]:
train_dataset = preprocessing(train=True)
dev_dataset = preprocessing(eval=True)
test_dataset = preprocessing(test=True)

In [7]:
def get_dataloader(text, metad, label):
    input_ids,token_type_ids,attention_mask = [],[],[]
    input_ids_meta,token_type_ids_meta,attention_mask_meta = [],[],[]
    labels = []
    for i,t in enumerate(text):
        if str(t)=='nan':
          t = ' '
        encoded = tokenizer.encode_plus(text=t,max_length=max_len,padding='max_length',truncation=True)
        # print(encoded)
        input_ids.append(encoded['input_ids'])
        # token_type_ids.append(encoded['token_type_ids'])
        attention_mask.append(encoded['attention_mask'])
        labels.append(int(label[i]))

        encoded_mata = tokenizer.encode_plus(text=metad[i],max_length=max_len_meta,padding='max_length',truncation=True)
        input_ids_meta.append(encoded_mata['input_ids'])
        # token_type_ids_meta.append(encoded_mata['token_type_ids'])
        attention_mask_meta.append(encoded_mata['attention_mask'])

    input_ids,attention_mask = torch.tensor(input_ids),torch.tensor(attention_mask)
    labels = torch.tensor(labels)
    input_ids_meta,attention_mask_meta = torch.tensor(input_ids_meta),torch.tensor(attention_mask_meta)
    data = TensorDataset(input_ids,attention_mask,labels,\
        input_ids_meta,attention_mask_meta)
    loader = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True) 
    return loader
    
train_loader = get_dataloader(train_dataset['statement'], get_meta_embed(train_dataset, meta_cols), label = train_dataset['label'])
dev_loader = get_dataloader(dev_dataset['statement'], get_meta_embed(dev_dataset, meta_cols), label = dev_dataset['label'])
test_loader = get_dataloader(test_dataset['statement'], get_meta_embed(test_dataset, meta_cols), label = test_dataset['label'])

In [8]:
class Roberta_Model(nn.Module):
    def __init__(self,config,model,bert_dir,classes=6):
        super(Roberta_Model,self).__init__()
        # self.config = config
        # self.roberta = model
        self.config = RobertaConfig.from_pretrained(bert_dir)
        self.roberta = RobertaModel.from_pretrained(bert_dir)
        for param in self.roberta.parameters():
            param.requires_grad=True
        print(self.config.hidden_size)
        self.fc = nn.Linear(self.config.hidden_size,classes)

    def forward(self,input_ids,attention_mask):
        output = self.roberta(input_ids,attention_mask)
        logit = self.fc(output[1])
        # print(logit.shape)
        return logit


In [9]:
def train(model,train_loader,valid_loader,optimizer,schedule,device,epoch, name = 'liar_roberta.pth'):
    print('device',device,'epoch',epoch,'train',len(train_loader),\
          'val',len(valid_loader), 'batch', BATCH_SIZE,'lr',LEARNING_RATE)
    best_acc = 0.0
    criterion = nn.CrossEntropyLoss()  
    for i in range(epoch):
        start = time.time()
        model.train()
        print("### Epoch {} ###".format(i+1))
        train_loss_sum = 0.0
        for idx,(ids,att,y,idm,attm) in enumerate(train_loader):
            ids,att,y,idm,attm = ids.to(device),att.to(device),y.to(device),idm.to(device),attm.to(device)
            ids = torch.concat((ids,idm),dim=1)
            att = torch.concat((att,attm),dim=1)
            y_pred = model(ids,att) 
            loss = criterion(y_pred,y) 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # schedule.step()
            train_loss_sum += loss.item()
            
            if(idx+1)%(len(train_loader)//5)==0:
                print("epoch {:04d}, step {:04d}/{:04d}, loss {:.4f}, time {:.4f}".format(
                i+1,idx+1,len(train_loader),train_loss_sum/(idx+1),time.time()-start))

        model.eval()
        acc = evaluate(model,valid_loader,device)  
        if acc > best_acc:
            best_acc = acc
            # torch.save(model.state_dict(),"best_roberta_model.pth") 
            torch.save(model.state_dict(), os.path.join(data_dir,name))
        print("current acc is {:.4f},best acc is {:.4f}".format(acc,best_acc))
        print("time costed = {}s \n".format(round(time.time()-start,5)))
   

In [10]:
model = Roberta_Model(config,bertmodel,bert_dir).to(device)
optimizer = AdamW(model.parameters(),lr=LEARNING_RATE,weight_decay=1e-4)
schedule = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=len(train_loader),num_training_steps=EPOCHS*len(test_loader))

train(model,train_loader,dev_loader,optimizer,schedule,device,5)

Some weights of the model checkpoint at /content/gdrive/MyDrive/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768
device cpu epoch 5 train 2561 val 321 batch 4 lr 8e-05
### Epoch 1 ###


KeyboardInterrupt: ignored

In [ ]:
# model = Roberta_Model(config,bertmodel,bert_dir).to(device)
# model.load_state_dict(torch.load(os.path.join(data_dir,'liar_roberta.pth')))
# for idx,(ids,att,y,idm,attm) in enumerate(train_loader):
#     ids,att,y,idm,attm = ids.to(device),att.to(device),y.to(device),idm.to(device),attm.to(device)
#     ids = torch.concat((ids,idm),dim=1)
#     att = torch.concat((att,attm),dim=1)
#     y_pred = model(ids,att)
#     print(y_pred)
#     categories = category_from_output(y_pred)
#     print(categories)
#     loss = criterion(y_pred,y)
#     break

In [ ]:
# model = Roberta_Model(config,bertmodel)
# model = model.to(device)
# evaluate(model, dev_loader,device)